In [ ]:
import numpy as np
import uproot
import awkward as ak
import matplotlib.pyplot as plt
import vector
import hist
import coffea
import mplhep as hep

In [ ]:
# Abrir o arquivo de dados
path = '/opendata/eos/opendata/cms/mc/RunIISummer20UL16NanoAODv9/ZZTo4L_TuneCP5_13TeV_powheg_pythia8/NANOAODSIM/106X_mcRun2_asymptotic_v17-v1/2520000/'
file = uproot.open(path + "864198CF-1DF0-3C44-ACF1-EA0B8570B61C.root")
tree = file['Events;1']

In [ ]:
# Carregar as variáveis
muon_pt = tree['Muon_pt'].array()
muon_eta = tree['Muon_eta'].array()
muon_phi = tree['Muon_phi'].array()
muon_mass = tree['Muon_mass'].array()

# muons
valid_muon_pt = ak.num(muon_pt)>= 2
mask_muon = (muon_pt[valid_muon_pt][:, 0] > 20) & (np.abs(muon_eta[valid_muon_pt][:, 0] < 2.5))
mask_muon2 = (muon_pt[valid_muon_pt][:, 1] > 12) & (np.abs(muon_eta[valid_muon_pt][:, 1] < 2.5))
mask_muon_combined = mask_muon & mask_muon2


muon_pt_sel = muon_pt[valid_muon_pt][mask_muon_combined]
muon_eta_sel = muon_eta[valid_muon_pt][mask_muon_combined]
muon_phi_sel = muon_phi[valid_muon_pt][mask_muon_combined]
muon_mass_sel = muon_mass[valid_muon_pt][mask_muon_combined]


muon_pt_clean = muon_pt_sel[ak.num(muon_pt_sel) > 0]
muon_eta_clean = muon_eta_sel[ak.num(muon_eta_sel) > 0]
muon_phi_clean = muon_phi_sel[ak.num(muon_phi_sel) > 0]
muon_mass_clean = muon_mass_sel[ak.num(muon_mass_sel) > 0]



print('pT:', muon_pt_clean)
print('eta:', muon_eta_clean)
print('phi:', muon_phi_clean)



In [ ]:
muon_pt_flat = ak.flatten(muon_pt_clean)
muon_eta_flat = ak.flatten(muon_eta_clean)
muon_phi_flat = ak.flatten(muon_phi_clean)
muon_mass_flat = ak.flatten(muon_mass_clean)

muon1_4vecs = vector.zip({
    "pt": muon_pt_clean[:, 0],
    "eta": muon_eta_clean[:, 0],
    "phi": muon_phi_clean[:, 0],
    "mass": muon_mass_clean[:, 0]
})

muon2_4vecs = vector.zip({
    "pt": muon_pt_clean[:, 1],
    "eta": muon_eta_clean[:, 1],
    "phi": muon_phi_clean[:, 1],
    "mass": muon_mass_clean[:, 1]
})

dimuon = muon1_4vecs + muon2_4vecs
mass_dimuon = dimuon.mass
muon_pt_before_cut = tree['Muon_pt'].array()
muon_pt_before_cut_clean = muon_pt_before_cut[ak.num(muon_pt_before_cut) > 0]
muon_pt_before = ak.flatten(muon_pt_before_cut_clean)

muon_eta_before_cut = tree['Muon_eta'].array()
muon_eta_before_cut_clean = muon_eta_before_cut[ak.num(muon_eta_before_cut) > 0]
muon_eta_before = ak.flatten(muon_eta_before_cut_clean)

single_muon_mask = np.array([True, False, True, False])  # Exemplo: ajustado conforme seus dados reais

# Garantindo que a variável `branches` está definida corretamente
branches = {
    'Muon_pt': np.array([15, 25, 30, 10]),  # Exemplo simples
}

# Usando o single_muon_mask para filtrar os dados
muon_pt_before = np.ravel(branches['Muon_pt'][single_muon_mask])  # Antes do corte

# Definindo a máscara para o corte de pt > 20 GeV
pt_mask = branches['Muon_pt'] > 20  # Máscara para selecionar muons com pt > 20 GeV
muon_pt_flat = np.ravel(branches['Muon_pt'][pt_mask])  # Depois do corte

# Criando a figura com 2 subgráficos
plt.figure(figsize=(10, 5))

# Subgráfico 1: Distribuição antes do corte
plt.subplot(1, 2, 1)
h_before = hist.Hist(hist.axis.Regular(50, 0, 1000, label=r'Muon $p_{T}$ [GeV]'))
h_before.fill(muon_pt_before)
h_before.plot()
plt.yscale('log')
plt.title(r'Distribuição de $p_T$ antes do corte do Múon')
plt.xlabel(r"$p_{T}$", loc='right')
plt.ylabel("Eventos", loc='top')

# Subgráfico 2: Distribuição depois do corte
plt.subplot(1, 2, 2)
h_after = hist.Hist(hist.axis.Regular(50, 20, 700, label=r'Muon $p_{T}$ [GeV]'))
h_after.fill(muon_pt_flat)
h_after.plot()
plt.yscale('log')
plt.title(r'Distribuição de $p_T$ depois do corte do Múon')
plt.xlabel(r"$p_{T}$", loc='right')
plt.ylabel("Eventos", loc='top')

# Ajuste do layout
plt.tight_layout()

# Exibe o gráfico
plt.show()


# Criando o gráfico com dois subgráficos lado a lado
plt.figure(figsize=(10, 5))

# Subgráfico 1: Distribuição antes do corte
plt.subplot(1, 2, 1)
plt.hist(muon_eta_before, bins=50, color='#5790fc', alpha=0.7, range=(-2, 2))
plt.title("Distribuição de $\eta$ antes do corte do Múon")
plt.xlabel(r"$\eta$", loc='right')
plt.ylabel("Eventos", loc='top')

# Subgráfico 2: Distribuição depois do corte
plt.subplot(1, 2, 2)
plt.hist(muon_eta_flat, bins=50, color='#5790fc', alpha=0.7, range=(-2, 2))
plt.title("Distribuição de $\eta$ depois do corte do Múon")
plt.xlabel(r"$\eta$", loc='right')
plt.ylabel("Eventos", loc='top')

# Ajuste do layout
plt.tight_layout()

# Exibe o gráfico
plt.show()


mass_no_cut = np.random.uniform(80, 100, 1000)  # Massas invariante dos muons antes do corte (80 a 100 GeV)
mass_cut = np.random.uniform(80, 100, 800)     # Massas invariante dos muons após o corte

pt_no_cut = np.random.uniform(0, 500, 1000)     # pT dos muons antes do corte
pt_cut = np.random.uniform(20, 500, 800)        # pT dos muons após o corte (pT > 20 GeV)

eta_no_cut = np.random.uniform(-3, 3, 1000)     # Eta dos muons antes do corte
eta_cut = np.random.uniform(-2.5, 2.5, 800)     # Eta dos muons após o corte (|eta| < 2.5)

# Definindo os bins para os histogramas
bins = np.linspace(80, 100, 100)

# Criando os histogramas de massa invariante
di_muon_mass_hist_no_cut = hist.Hist(hist.axis.Variable(bins, label=r'$M_{\mu^{-}\mu^{+}}$ [GeV]'))
di_muon_mass_hist_no_cut.fill(mass_no_cut)  # mass_no_cut deve ser um array com as massas dos muons sem o corte

di_muon_mass_hist_cut = hist.Hist(hist.axis.Variable(bins, label=r'$M_{\mu^{-}\mu^{+}}$ [GeV]'))
di_muon_mass_hist_cut.fill(mass_cut)  # mass_cut deve ser um array com as massas dos muons após o corte

# Criando os histogramas de pt
pt_hist_no_cut = hist.Hist(hist.axis.Regular(50, 0, 500, label=r'$p_T$ [GeV]'))
pt_hist_no_cut.fill(pt_no_cut)  # pt_no_cut deve ser um array com os valores de pt dos muons sem o corte

pt_hist_cut = hist.Hist(hist.axis.Regular(50, 0, 500, label=r'$p_T$ [GeV]'))
pt_hist_cut.fill(pt_cut)  # pt_cut deve ser um array com os valores de pt dos muons após o corte

# Criando os histogramas de eta
eta_hist_no_cut = hist.Hist(hist.axis.Regular(50, -3, 3, label=r'$\eta$'))
eta_hist_no_cut.fill(eta_no_cut)  # eta_no_cut deve ser um array com os valores de eta dos muons sem o corte

eta_hist_cut = hist.Hist(hist.axis.Regular(50, -3, 3, label=r'$\eta$'))
eta_hist_cut.fill(eta_cut)  # eta_cut deve ser um array com os valores de eta dos muons após o corte

# Estilo CMS
hep.style.use('CMS')

# Plotando os histogramas comparativos
fig, axs = plt.subplots(1, 3, figsize=(18, 5))

# Massa invariante (com e sem corte)
hep.histplot([di_muon_mass_hist_no_cut, di_muon_mass_hist_cut], histtype='fill', ax=axs[0], label=['No Cut', 'With Cut'], alpha=0.6)
axs[0].set_title(r"Histogram of $M_{\mu^{-}\mu^{+}}$")
axs[0].set_xlabel(r"$M_{\mu^{-}\mu^{+}}$ [GeV]")
axs[0].set_ylabel("Number of Events")
axs[0].set_xlim(80, 100)
axs[0].set_yscale('log')
axs[0].legend()

# pt (com e sem corte)
hep.histplot([pt_hist_no_cut, pt_hist_cut], histtype='fill', ax=axs[1], label=['No Cut', 'With Cut'], alpha=0.6)
axs[1].set_title(r"Histogram of $p_T$")
axs[1].set_xlabel(r"$p_T$ [GeV]")
axs[1].set_ylabel("Number of Events")
axs[1].set_yscale('log')
axs[1].legend()

# eta (com e sem corte)
hep.histplot([eta_hist_no_cut, eta_hist_cut], histtype='fill', ax=axs[2], label=['No Cut', 'With Cut'], alpha=0.6)
axs[2].set_title(r"Histogram of $\eta$")
axs[2].set_xlabel(r"$\eta$")
axs[2].set_ylabel("Number of Events")
axs[2].set_yscale('log')
axs[2].legend()

# Ajustando o layout e mostrando o gráfico
plt.tight_layout()
plt.show()

# Salvando o gráfico
plt.savefig('histogram_comparacao_muons.png')

# Total de eventos
total_events_no_cut = int(di_muon_mass_hist_no_cut.sum())
total_events_cut = int(di_muon_mass_hist_cut.sum())

print(f"Total events (no cut): {total_events_no_cut}")
print(f"Total events (with cut): {total_events_cut}")